Họ và tên : Hà Quang Vinh

Mã sinh viên : 22174600065

Lớp : DHKL16A2HN

BÀI TẬP SQL CHƯƠNG 3



#  Câu 1: viết câu lệnh SQL để tính sự tương quan giữa A và B theo công thức


In [12]:
import sqlite3
import math

# Tạo kết nối SQLite
conn = sqlite3.connect(':memory:')

# Đăng ký SQRT và POWER vào SQLite
conn.create_function("SQRT", 1, math.sqrt)
conn.create_function("POWER", 2, math.pow)

# Tạo bảng và dữ liệu
cursor = conn.cursor()
cursor.execute('CREATE TABLE data (a REAL, b REAL)')
cursor.executemany('INSERT INTO data (a, b) VALUES (?, ?)', [
    (1, 2),
    (2, 3),
    (3, 6),
    (4, 8),
    (5, 10)
])
conn.commit()
query = """
SELECT 
    (COUNT(*) * SUM(a * b) - SUM(a) * SUM(b)) / 
    (SQRT(COUNT(*) * SUM(a * a) - SUM(a) * SUM(a)) * 
     SQRT(COUNT(*) * SUM(b * b) - SUM(b) * SUM(b))) AS correlation
FROM data;
"""

result = cursor.execute(query).fetchone()
print("Hệ số tương quan Pearson giữa A và B:", result[0])


Hệ số tương quan Pearson giữa A và B: 0.9921567416492215


#  Câu 2:

In [ ]:
import sqlite3
import pandas as pd
from scipy.stats import chi2_contingency

# Kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng và thêm dữ liệu dạng rộng (Day, A, B, C)
cursor.execute("""
CREATE TABLE scores (
    Day TEXT,
    A REAL,
    B REAL,
    C REAL
)
""")

# Dữ liệu gốc
data = [
    ("Day 1", 8,   9,   7),
    ("Day 2", 7.5, 8.5, 7),
    ("Day 3", 6,   7,   8),
    ("Day 4", 7,   6,   5)
]
cursor.executemany("INSERT INTO scores VALUES (?, ?, ?, ?)", data)
conn.commit()

# Chuyển về long format bằng SQL: Day | Model | Score
query_long = """
SELECT Day, 'A' AS Model, A AS Score FROM scores
UNION ALL
SELECT Day, 'B' AS Model, B AS Score FROM scores
UNION ALL
SELECT Day, 'C' AS Model, C AS Score FROM scores
"""
df_long = pd.read_sql_query(query_long, conn)

# Phân loại điểm thành nhóm: Low / Medium / High
def classify(score):
    if score <= 6:
        return 'Low'
    elif score < 8:
        return 'Medium'
    else:
        return 'High'

df_long['Score_Level'] = df_long['Score'].apply(classify)


contingency_model = pd.crosstab(df_long['Model'], df_long['Score_Level'])
chi2_m, p_m, _, _ = chi2_contingency(contingency_model)


contingency_day = pd.crosstab(df_long['Day'], df_long['Score_Level'])
chi2_d, p_d, _, _ = chi2_contingency(contingency_day)

# In kết quả
print("Chi-square theo mẫu xe:")
print(contingency_model)
print(f"Chi2 = {chi2_m:.4f} | p-value = {p_m:.4f}\n")

print("Chi-square theo ngày:")
print(contingency_day)
print(f"Chi2 = {chi2_d:.4f} | p-value = {p_d:.4f}")


Chi-square theo mẫu xe:
Score_Level  High  Low  Medium
Model                         
A               1    1       2
B               2    1       1
C               1    1       2
Chi2 = 0.9000 | p-value = 0.9246

Chi-square theo ngày:
Score_Level  High  Low  Medium
Day                           
Day 1           2    0       1
Day 2           1    0       2
Day 3           1    1       1
Day 4           0    2       1
Chi2 = 6.2667 | p-value = 0.3940


-Nhận xét: Chi-square theo mẫu xe
    
Vì p-value = 0.9246 > 0.05, ta không bác bỏ giả thuyết H0.

Không có sự khác biệt đáng kể giữa các mẫu A, B, C về mặt điểm số trong 4 ngày thử nghiệm.

Kết luận: Kết quả không phụ thuộc vào mẫu xe.

-Nhận xét: Chi-square theo ngày

Vì p-value = 0.3940 > 0.05, ta không bác bỏ giả thuyết H0.

Không có sự khác biệt đáng kể giữa các ngày thử nghiệm về mặt điểm số.

Kết luận: Kết quả không phụ thuộc vào ngày.

-> Kết quả thử nghiệm không phụ thuộc vào mẫu xe và cũng không phụ thuộc vào ngày.

#  Câu 3: Bảng flights(departure_time,…) chứa các giá trị thời gian dưới dạng số nguyên (ví dụ: 830 cho 8:30AM, 1445 cho 2:45 PM). Hãy chuyển đổi các giá trị này thành định dạng thời gian.


In [17]:
import sqlite3
import pandas as pd

# Kết nối SQLite (trong bộ nhớ)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng flights và thêm dữ liệu mẫu
cursor.execute("""
CREATE TABLE flights (
    id INTEGER PRIMARY KEY,
    departure_time INTEGER
)
""")

# Dữ liệu mẫu
sample_data = [
    (1, 830),
    (2, 1445),    
    (3, 5),       
    (4, 0),       
    (5, 1200),   
    (6, 945),     
    (7, 1310)    
]

# Thêm dữ liệu vào bảng
cursor.executemany("INSERT INTO flights (id, departure_time) VALUES (?, ?)", sample_data)
conn.commit()

# Truy vấn: chuyển đổi departure_time sang định dạng HH:MM
query = """
SELECT 
    id,
    departure_time,
    printf('%02d:%02d', departure_time / 100, departure_time % 100) AS formatted_time
FROM flights
"""

# Đọc kết quả bằng pandas và hiển thị
df = pd.read_sql_query(query, conn)
print(df)


   id  departure_time formatted_time
0   1             830          08:30
1   2            1445          14:45
2   3               5          00:05
3   4               0          00:00
4   5            1200          12:00
5   6             945          09:45
6   7            1310          13:10


#  Câu 4: Viết truy vấn SQL để tìm các ngoại lệ bằng cách sử dụng MAD. Một quy tắc chung là xem xét các giá trị ngoại lệ lớn hơn 1,5 lần so với giá trị MAD, trong đó x là số độ lệch chuẩn mà ta coi là có ý nghĩa.

In [ ]:
import sqlite3
import pandas as pd

# Kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng và thêm dữ liệu mẫu
cursor.execute("""
CREATE TABLE data (
    id INTEGER PRIMARY KEY,
    value REAL
)
""")
values = [10, 12, 13, 11, 15, 14, 9, 200]  # 200 là ngoại lệ
cursor.executemany("INSERT INTO data (value) VALUES (?)", [(v,) for v in values])
conn.commit()

# Bước 1: Lấy giá trị và tính median
df = pd.read_sql_query("SELECT * FROM data", conn)
median = df['value'].median()

# Bước 2: Tạo bảng tạm tính |value - median|
df['abs_dev'] = abs(df['value'] - median)
df[['id', 'abs_dev']].to_sql("abs_dev", conn, if_exists='replace', index=False)

# Bước 3: Tính MAD (median của abs_dev)
mad = df['abs_dev'].median()

# Bước 4: Tìm các điểm có |value - median| > 1.5 * MAD bằng SQL
threshold = 1.5 * mad
query = f"""
SELECT d.id, d.value
FROM data d
JOIN abs_dev a ON d.id = a.id
WHERE a.abs_dev > {threshold}
"""

outliers = pd.read_sql_query(query, conn)
print("Các giá trị ngoại lệ theo MAD:")
print(outliers)


Các giá trị ngoại lệ theo MAD:
   id  value
0   7    9.0
1   8  200.0


#  Câu 5: Hãy xác định liệu hai người trong bảng Patient(last_name, weight, height) có phải là một người hay không bằng cách sử dụng khoảng cách kết hợp Boolean trên “last_name” và “weight”.

In [20]:
import sqlite3
import pandas as pd

# Tạo kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng Patient và thêm dữ liệu mẫu
cursor.execute("""
CREATE TABLE Patient (
    last_name TEXT,
    weight INTEGER,
    height REAL
)
""")

patients = [
    ('Ha', 60, 165),
    ('Nguyen', 60, 168),  # cùng họ và cân nặng → có thể cùng người
    ('Tran', 55, 160),
    ('Le', 60, 170),
    ('Nguyen', 58, 165)   # gần giống nhưng cân nặng khác
]
cursor.executemany("INSERT INTO Patient VALUES (?, ?, ?)", patients)
conn.commit()

# Thực hiện truy vấn so sánh khoảng cách Boolean
query = """
SELECT 
    p1.last_name AS last_name1, p2.last_name AS last_name2,
    p1.weight AS weight1, p2.weight AS weight2,
    CASE WHEN p1.last_name = p2.last_name THEN 0 ELSE 1 END AS last_name_dist,
    CASE WHEN p1.weight = p2.weight THEN 0 ELSE 1 END AS weight_dist,
    (CASE WHEN p1.last_name = p2.last_name THEN 0 ELSE 1 END + 
     CASE WHEN p1.weight = p2.weight THEN 0 ELSE 1 END) AS total_distance,
    CASE 
        WHEN (p1.last_name = p2.last_name AND p1.weight = p2.weight) THEN 'Same person'
        ELSE 'Different'
    END AS result
FROM Patient p1
JOIN Patient p2 ON p1.rowid < p2.rowid
"""

# Lấy kết quả và hiển thị
df_result = pd.read_sql_query(query, conn)
print(df_result)


  last_name1 last_name2  weight1  weight2  last_name_dist  weight_dist  \
0         Ha     Nguyen       60       60               1            0   
1         Ha       Tran       60       55               1            1   
2         Ha         Le       60       60               1            0   
3         Ha     Nguyen       60       58               1            1   
4     Nguyen       Tran       60       55               1            1   
5     Nguyen         Le       60       60               1            0   
6     Nguyen     Nguyen       60       58               0            1   
7       Tran         Le       55       60               1            1   
8       Tran     Nguyen       55       58               1            1   
9         Le     Nguyen       60       58               1            1   

   total_distance     result  
0               1  Different  
1               2  Different  
2               1  Different  
3               2  Different  
4               2  Different  